# Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [21]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [22]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gemini-1.5'
- Thank you for your understanding!

In [23]:

import os

with open('env.txt', 'r') as file:
    serper_api_key = file.read().strip()

os.environ["SERPER_API_KEY"] = serper_api_key

In [24]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

FOLDER_NAME="."

## crewAI Tools

In [25]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [26]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model = "gemini-1.5-pro-002",
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    return(responses.text)

## Creating Agents

In [130]:
gym_agent = Agent(
    role="Gym workout planner",
    goal="creating personalized workout plans "
         "based on user profile.",
    backstory="""
                You are a fitness expert, known for creating personalized workout plans.
You transform complex concepts into compelling narratives. 
Make sure to include the following:

Frequency
Duration
Intensity
Equipment
Focus Areas
Daily Structure
Weekly Summary

frequency: 3 days a week (Monday, Wednesday, Friday)
duration: 30 minutes per session
intensity: Low
equipment: Dumbbells
focus_areas: Upper Body, Lower Body
daily_structure: 

  Monday: 
    duration: 30 minutes
    warm_up: 
      duration: 5 minutes
      exercises: 
        name: Arm Circles, duration: 30 seconds
        name: Leg Swings, duration: 1 minute
        name: March in Place, duration: 3.5 minutes
    main_workout: 
      duration: 20 minutes
      circuit: 
        exercise_duration: 30 seconds
        rest_duration: 30 seconds
        repeat: 2 times
        exercises: 
          Dumbbell Shoulder Press
          Dumbbell Bent Over Rows
          Dumbbell Goblet Squats
          Dumbbell Bicep Curls
          Dumbbell Tricep Extensions
    cool_down: 
      duration: 5 minutes
      exercises: 
        name: Shoulder Stretch, duration: 30 seconds per arm
        name: Standing Quad Stretch, duration: 30 seconds per leg
        name: Deep Breathing, duration: 2 minutes
  Wednesday: 
    duration: 30 minutes
    warm_up: 
      duration: 5 minutes
      exercises: 
        name: Side Lunges, duration: 1 minute
        name: Arm Circles, duration: 30 seconds
        name: High Knees, duration: 3.5 minutes
    main_workout: 
      duration: 20 minutes
      circuit: 
        exercise_duration: 30 seconds
        rest_duration: 30 seconds
        repeat: 2 times
        exercises: 
          Dumbbell Chest Press
          Dumbbell Deadlifts
          Dumbbell Lateral Raises
          Dumbbell Front Raises
          Standing Calf Raises
    cool_down: 
      duration: 5 minutes
      exercises: 
        name: Arm Across Chest Stretch, duration: 30 seconds per arm
        name: Standing Hamstring Stretch, duration: 30 seconds per leg
        name: Neck Stretch, duration: 30 seconds
        name: Deep Breathing, duration: 2 minutes
  Friday: 
    duration: 30 minutes
    warm_up: 
      duration: 5 minutes
      exercises: 
        name: Dynamic Hip Flexor Stretch, duration: 1 minute
        name: Arm Circles, duration: 30 seconds
        name: Jumping Jacks, duration: 3.5 minutes
    main_workout: 
      duration: 20 minutes
      circuit: 
        exercise_duration: 30 seconds
        rest_duration: 30 seconds
        repeat: 2 times
        exercises: 
          Dumbbell Push Press
          Dumbbell Step-Ups
          Dumbbell Russian Twists
          Dumbbell Sumo Squats
          Plank Hold
    cool_down: 
      duration: 5 minutes
      exercises: 
        name: Child’s Pose, duration: 1 minute
        name: Seated Forward Bend, duration: 1 minute
        name: Side Stretch, duration: 30 seconds per side
        name: Deep Breathing, duration: 2 minutes

weekly_summary: 
  Monday: Upper Body & Lower Body Workout A
  Wednesday: Upper Body & Lower Body Workout B
  Friday: Upper Body & Lower Body Workout C
  Saturday: Rest Days / Optional light activity like walking or stretching
  Sunday: 

This is an example of what you should fill in and return. 
Don't add any other properties, only these. 
It can be in markdown format. 
            """,
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [142]:
meal_agent = Agent(
    role="Meal Planning Agent",
    goal="Develop  various meal plans  based "
         "on user profile and gym routine.",
    backstory="""
                You are a fitness expert, known for creating personalized meal plans.
You transform complex concepts into compelling narratives. 
Make sure to include the following:

Nutritional Goals
Meal Timing
Weekly Meals
Additional Tips

meal_plan: 
  nutritional_goals: 
    protein: Lean meats, fish, eggs, legumes
    carbohydrates: Whole grains, fruits, vegetables
    fats: Avocado, nuts, olive oil
  meal_timing: Breakfast, Lunch, Dinner, and Snacks around workout sessions
  weekly_meals: 
    Monday: 
      Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts
      Snack: A banana
      Lunch: Grilled chicken salad with mixed greens, cherry tomatoes, cucumbers, and balsamic dressing
      Snack: Hummus with carrot sticks
      Dinner: Baked salmon with quinoa and steamed broccoli
    Tuesday: 
      Breakfast: Oatmeal topped with sliced apple and cinnamon
      Snack: A handful of almonds
      Lunch: Turkey wrap with whole grain tortilla, lettuce, tomato, and mustard
      Snack: Cottage cheese with pineapple chunks
      Dinner: Stir-fried tofu with mixed vegetables and brown rice
    Wednesday: 
      break: Rest Day
      Breakfast: Smoothie with spinach, banana, protein powder, and almond milk
      Snack: Hard-boiled eggs
      Lunch: Quinoa salad with chickpeas, bell peppers, and lemon vinaigrette
      Snack: Rice cakes with peanut butter
      Dinner: Roast chicken with sweet potatoes and green beans
    Thursday: 
      Breakfast: Whole grain toast with avocado and poached egg
      Snack: Greek yogurt with honey and walnuts
      Lunch: Lentil soup with a side salad
      Snack: Sliced cucumbers with tzatziki
      Dinner: Grilled shrimp tacos with cabbage slaw
    Friday: 
      Breakfast: Chia seed pudding made with almond milk and topped with fresh fruit
      Snack: Protein bar
      Lunch: Spinach and feta omelet with whole grain toast
      Snack: A small apple with almond butter
      Dinner: Ground turkey stuffed peppers
  additional_tips: 
    Stay hydrated and drink plenty of water throughout the day
    Adjust portion sizes based on personal needs and weight loss goals
    Consider incorporating a variety of spices and herbs to enhance flavor and nutrition

This is an example of what you should fill in and return. 
Don't add any other properties, only these. 
It can be in markdown format.
For the nutritional_goals property, you can include other things, and this can differ from person to person. 
Take into account their age, gender, and fitness level.
            """,
    
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [153]:
writer_agent = Agent(
    role="Writer Agent",
    goal="Writes routines for Gym and meals "
         "based on approved health strategies.",
    backstory="""
workout_plan: 
  frequency: 3 days a week (Monday, Wednesday, Friday)
  duration: 30 minutes per session
  intensity: Low
  equipment: Dumbbells
  focus_areas: Upper Body, Lower Body
  daily_structure: 
    Monday: 
      duration: 30 minutes
      warm_up: 
        duration: 5 minutes
        exercises: 
          name: Arm Circles, duration: 30 seconds
          name: Leg Swings, duration: 1 minute
          name: March in Place, duration: 3.5 minutes
      main_workout: 
        duration: 20 minutes
        circuit: 
          exercise_duration: 30 seconds
          rest_duration: 30 seconds
          repeat: 2 times
          exercises: 
            Dumbbell Shoulder Press
            Dumbbell Bent Over Rows
            Dumbbell Goblet Squats
            Dumbbell Bicep Curls
            Dumbbell Tricep Extensions
      cool_down: 
        duration: 5 minutes
        exercises: 
          name: Shoulder Stretch, duration: 30 seconds per arm
          name: Standing Quad Stretch, duration: 30 seconds per leg
          name: Deep Breathing, duration: 2 minutes
    Wednesday: 
      duration: 30 minutes
      warm_up: 
        duration: 5 minutes
        exercises: 
          name: Side Lunges, duration: 1 minute
          name: Arm Circles, duration: 30 seconds
          name: High Knees, duration: 3.5 minutes
      main_workout: 
        duration: 20 minutes
        circuit: 
          exercise_duration: 30 seconds
          rest_duration: 30 seconds
          repeat: 2 times
          exercises: 
            Dumbbell Chest Press
            Dumbbell Deadlifts
            Dumbbell Lateral Raises
            Dumbbell Front Raises
            Standing Calf Raises
      cool_down: 
        duration: 5 minutes
        exercises: 
          name: Arm Across Chest Stretch, duration: 30 seconds per arm
          name: Standing Hamstring Stretch, duration: 30 seconds per leg
          name: Neck Stretch, duration: 30 seconds
          name: Deep Breathing, duration: 2 minutes
    Friday: 
      duration: 30 minutes
      warm_up: 
        duration: 5 minutes
        exercises: 
          name: Dynamic Hip Flexor Stretch, duration: 1 minute
          name: Arm Circles, duration: 30 seconds
          name: Jumping Jacks, duration: 3.5 minutes
      main_workout: 
        duration: 20 minutes
        circuit: 
          exercise_duration: 30 seconds
          rest_duration: 30 seconds
          repeat: 2 times
          exercises: 
            Dumbbell Push Press
            Dumbbell Step-Ups
            Dumbbell Russian Twists
            Dumbbell Sumo Squats
            Plank Hold
      cool_down: 
        duration: 5 minutes
        exercises: 
          name: Childs Pose, duration: 1 minute
          name: Seated Forward Bend, duration: 1 minute
          name: Side Stretch, duration: 30 seconds per side
          name: Deep Breathing, duration: 2 minutes
  weekly_summary: 
    Monday: Upper Body & Lower Body Workout A
    Wednesday: Upper Body & Lower Body Workout B
    Friday: Upper Body & Lower Body Workout C
    Saturday: Rest Days / Optional light activity like walking or stretching
    Sunday: 
meal_plan: 
  nutritional_goals: 
    protein: Lean meats, fish, eggs, legumes
    carbohydrates: Whole grains, fruits, vegetables
    fats: Avocado, nuts, olive oil
  meal_timing: Breakfast, Lunch, Dinner, and Snacks around workout sessions
  weekly_meals: 
    Monday: 
      Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts
      Snack: A banana
      Lunch: Grilled chicken salad with mixed greens, cherry tomatoes, cucumbers, and balsamic dressing
      Snack: Hummus with carrot sticks
      Dinner: Baked salmon with quinoa and steamed broccoli
    Tuesday: 
      Breakfast: Oatmeal topped with sliced apple and cinnamon
      Snack: A handful of almonds
      Lunch: Turkey wrap with whole grain tortilla, lettuce, tomato, and mustard
      Snack: Cottage cheese with pineapple chunks
      Dinner: Stir-fried tofu with mixed vegetables and brown rice
    Wednesday: 
      break: Rest Day
      Breakfast: Smoothie with spinach, banana, protein powder, and almond milk
      Snack: Hard-boiled eggs
      Lunch: Quinoa salad with chickpeas, bell peppers, and lemon vinaigrette
      Snack: Rice cakes with peanut butter
      Dinner: Roast chicken with sweet potatoes and green beans
    Thursday: 
      Breakfast: Whole grain toast with avocado and poached egg
      Snack: Greek yogurt with honey and walnuts
      Lunch: Lentil soup with a side salad
      Snack: Sliced cucumbers with tzatziki
      Dinner: Grilled shrimp tacos with cabbage slaw
    Friday: 
      Breakfast: Chia seed pudding made with almond milk and topped with fresh fruit
      Snack: Protein bar
      Lunch: Spinach and feta omelet with whole grain toast
      Snack: A small apple with almond butter
      Dinner: Ground turkey stuffed peppers
  additional_tips: 
    Stay hydrated and drink plenty of water throughout the day
    Adjust portion sizes based on personal needs and weight loss goals
    Consider incorporating a variety of spices and herbs to enhance flavor and nutrition
                """,
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [154]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [155]:
#         workout_task = f"""Create a workout plan for me as I am a beginner. Here is information about me:
#         {user_info}"""
        
#         meal_plan_task = """Create a meal plan for me based on the workout plan."""
        
#         writing_task = """Take the workout plan and the meal plan and write them neatly for the user. Make the output in json format."""


In [156]:
# Task for Data Analyst Agent: Analyze Market Data
gym_task = Task(
    description=(
            "Create a workout plan for me as I am a beginner. Here is information about me "
    ),
    expected_output=(
        "Good Gym workout stratergies "
        "plan a workout routine thats is wholesome and doable in 30-35mins each day."
    ),
    agent=gym_agent,
)

In [157]:
# Task for Trading Strategy Agent: Develop Trading Strategies
meal_task = Task(
    description=(
        "Create a meal plan for me based on the workout plan."
    ),
    expected_output=(
        "A set of potential meal plan based on user profile "
    ),
    agent=meal_agent,
)


In [158]:
# Task for Trade Advisor Agent: Plan Trade Execution
writer_task = Task(
    description=(
        "Take the workout plan and the meal plan and write them neatly for the user. Make the output in json format."
    ),
    expected_output=(
        "Detailed write up  plans suggesting how and when to "
        "do the gym and meal suggestion."
    ),
    agent=writer_agent,
)


In [159]:
# # Task for Risk Advisor Agent: Assess Trading Risks
# risk_assessment_task = Task(
#     description=(
#         "Evaluate the risks associated with the proposed trading "
#         "strategies and execution plans for {stock_selection}. "
#         "Provide a detailed analysis of potential risks "
#         "and suggest mitigation strategies."
#     ),
#     expected_output=(
#         "A comprehensive risk analysis report detailing potential "
#         "risks and mitigation recommendations for {stock_selection}."
#     ),
#     agent=risk_management_agent,
# )

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [160]:
from crewai import Crew, Process

# Define the crew with agents and tasks
health_crew = Crew(
    agents=[gym_agent, 
            meal_agent, 
            writer_agent
            ],
    
    tasks=[gym_task, 
           meal_task, 
           writer_task 
           ],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

2024-10-22 08:55:30,925 - 139929801139392 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [161]:
# Example data for kicking off the process
user_info = {
    "Age": '34', 
    'Gender': 'Male', 
    'Fitness Level': 'Beginner', 
    'Goals': 'Weight Loss', 
    'Workout Duration': '15 minutes', 
    'Workout Type': 'Cardio', 
    'Equipment': 'Dumbbells', 
    'Workout Days': '5 days per week', 
    'Intensity': 'Low', 
    'Preferred Time': 'Morning', 
    'Warm-up/Cool-down': 'Yes', 
    'Focus Areas': 'Upper Body'
}


**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [162]:
### this execution will take some time to run
result = health_crew.kickoff(inputs=user_info)

# Agent: Crew Manager
## Task: Create a workout plan for me as I am a beginner. Here is information about me 
# Agent: Gym workout planner
## Task: Create a workout plan for a beginner.


# Agent: Gym workout planner
## Final Answer: 
```markdown
# Beginner's Wholesome & Doable Workout Plan

This plan is designed for complete beginners, focusing on building a foundation of strength and endurance.  It's designed to be completed in about 30 minutes, 3 times a week.  Remember to listen to your body and rest when needed.

**Frequency:** 3 days a week (Monday, Wednesday, Friday)
**Duration:** ~30 minutes per session
**Intensity:** Low to Moderate
**Equipment:**  Bodyweight (Optional: Mat, light dumbbells)

**Focus Areas:** Full Body

**Daily Structure:**

**Monday:**

* **Warm-up (5 minutes):**
    * Jumping Jacks (30 seconds)
    * High Knees (30 seconds)
    * Butt Kicks (30 seconds)
    * Arm Circles (forward and backward, 30 seconds each)
    * Dynamic Stretching: Leg swings (forward an

- Display the final result as Markdown.

In [163]:
# from IPython.display import Markdown
# Markdown(result)

from IPython.display import Markdown
Markdown(result.raw)

```json
{
  "workout_plan": {
    "frequency": "3 days a week (Monday, Wednesday, Friday)",
    "duration": "30 minutes per session",
    "intensity": "Low",
    "equipment": "Dumbbells",
    "focus_areas": ["Upper Body", "Lower Body"],
    "daily_structure": {
      "Monday": {
        "duration": "30 minutes",
        "warm_up": {
          "duration": "5 minutes",
          "exercises": [
            {"name": "Arm Circles", "duration": "30 seconds"},
            {"name": "Leg Swings", "duration": "1 minute"},
            {"name": "March in Place", "duration": "3.5 minutes"}
          ]
        },
        "main_workout": {
          "duration": "20 minutes",
          "circuit": {
            "exercise_duration": "30 seconds",
            "rest_duration": "30 seconds",
            "repeat": 2,
            "exercises": [
              "Dumbbell Shoulder Press",
              "Dumbbell Bent Over Rows",
              "Dumbbell Goblet Squats",
              "Dumbbell Bicep Curls",
              "Dumbbell Tricep Extensions"
            ]
          }
        },
        "cool_down": {
          "duration": "5 minutes",
          "exercises": [
            {"name": "Shoulder Stretch", "duration": "30 seconds per arm"},
            {"name": "Standing Quad Stretch", "duration": "30 seconds per leg"},
            {"name": "Deep Breathing", "duration": "2 minutes"}
          ]
        }
      },
      "Wednesday": {
        // ... (Wednesday workout details)
      },
      "Friday": {
        // ... (Friday workout details)
      }
    },
    "weekly_summary": {
      "Monday": "Upper Body & Lower Body Workout A",
      "Wednesday": "Upper Body & Lower Body Workout B",
      "Friday": "Upper Body & Lower Body Workout C",
      "Saturday": "Rest Days / Optional light activity like walking or stretching",
      "Sunday": ""
    }
  },
  "meal_plan": {
    "nutritional_goals": {
      "protein": "Lean meats, fish, eggs, legumes",
      "carbohydrates": "Whole grains, fruits, vegetables",
      "fats": "Avocado, nuts, olive oil"
    },
    "meal_timing": "Breakfast, Lunch, Dinner, and Snacks around workout sessions",
    "weekly_meals": {
      "Monday": {
        "Breakfast": "Greek yogurt with mixed berries and a sprinkle of nuts",
        "Snack": "A banana",
        "Lunch": "Grilled chicken salad with mixed greens, cherry tomatoes, cucumbers, and balsamic dressing",
        "Snack": "Hummus with carrot sticks",
        "Dinner": "Baked salmon with quinoa and steamed broccoli"
      },
      "Tuesday": {
        // ... (Tuesday meal details)
      },
      "Wednesday": {
          "break": "Rest Day",
        "Breakfast": "Smoothie with spinach, banana, protein powder, and almond milk",
        "Snack": "Hard-boiled eggs",
        "Lunch": "Quinoa salad with chickpeas, bell peppers, and lemon vinaigrette",
        "Snack": "Rice cakes with peanut butter",
        "Dinner": "Roast chicken with sweet potatoes and green beans"
      },
      "Thursday": {
        // ... (Thursday meal details)
      },
      "Friday": {
        // ... (Friday meal details)
      }


    },
    "additional_tips": {
      "hydration": "Stay hydrated and drink plenty of water throughout the day",
      "portion_control": "Adjust portion sizes based on personal needs and weight loss goals",
      "flavor_nutrition": "Consider incorporating a variety of spices and herbs to enhance flavor and nutrition"
    }
  }
}
```